In [1]:
import os
import sys
parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from src.base_functions import turn_voltage

The following code snippets try to check the correctness of the transformer parameters, defined by doi.org/10.1088/0953-2048/20/5/010

The turn voltage calculated in 2 different ways:
   - from the core parameters
   - from the given line/phase voltages and turn number of the windings

The given parameters of the transformer core are:
 - the core diameter is 396 mm, the core radius is 198 mm
 - the flux density in the core is 1.275 T
 - the filling factor of the core was not given, we calculating by a usual 88 %
 - the network frequency is 50 Hz
 
Using the turn voltage base function of the library:

In [16]:
ut = turn_voltage(1.275, 198.0, 0.88, 50.0)
print('turn voltage:',round(ut,2), 'V')

turn voltage: 30.68 V


Turn voltage can be calculated from the winding parameters of the star/star connected three phase transformer:

In [14]:
Np = 262.0 # turns in the primary winding
Up = 10500 # V 
utp = Up/Np/1.73
print("Turn voltage from primary winding data: ", round(utp,2), "V")

# secondary winding
Ns = 10
Us = 400
uts= Us/1.73/Ns
print("Turn voltage from secondary winding data:",round(uts,2), "V")

Turn voltage from primary winding data:  23.17 V
Turn voltage from secondary winding data: 23.12 V


Therefore, some parameter of the core is wrong, maybe the flux density of the core is a bit lower in the practice, in this case the flux density should be 0.963 T

In [4]:
ut = turn_voltage(1.275, 198.0, 0.88, 50.0)
print(ut)

30.677932291307098


In [5]:
from importlib_resources import files
from src.two_winding_model import TransformerDesign, TwoWindingModel

path = files("data").joinpath("630kVA_sc_transformer.json")

import json

with open(path) as json_file:
    data = json.load(json_file)

transformer = TransformerDesign.from_dict(data)

trafo_model = TwoWindingModel(input=transformer)
trafo_model.calculate(is_sc=True)

# FEM calculation
trafo_model.fem_simulation()

Mesh Generator: Initial mesh generation
Mesh generator: Triangle
Mesh Generator: Reading initial mesh from memory
Solver: Solver - MUMPS (none)
Solver: Elapsed time: 00:01.707 s
0.02302695201853413 0.016638417127457783


The following command can list the main results of the calculation:

In [6]:
print(trafo_model.results)
print('analytical sci:', trafo_model.results.sci)

MainResults(wh=809.5, feasible=True, core_loss=2.4095115858526066, load_loss=275997499249627.62, copper_mass=0.0, sci=0.016699122467986466, window_width=139.24546492552605, core_mass=4981.568642674868, turn_voltage=30.677932291307098, capitalized_cost=0.0, fem_based_sci=2.2771026207976126, fem_bax=0.02302695201853413, fem_brad=0.016638417127457783)
analytical sci: 0.016699122467986466


The resulting fem-based short circuit impedance is 2.28 %, which is very close to the given 2.45 %, however, the analytically calculated sci is only 1.7%. The reason is that the analytical formula contains the turn voltage, which is different, the core somehow not sized properly. 

In [7]:
print(trafo_model.hv_winding)

WindingDesign(inner_radius=288.5381603296059, thickness=10.207304595920151, winding_height=354.83, filling_factor=50.0, current_density=3.78, mass=64.15055287405289, dc_loss=0.0, ac_loss=10357905069120.916, outer_radius=298.74546492552605, cable_length=7886328785657.566)


In [8]:
print(trafo_model.lv_winding)

WindingDesign(inner_radius=244.0, thickness=6.038160329605889, winding_height=342.5, filling_factor=50.0, current_density=6.62, mass=30.8139105809291, dc_loss=0.0, ac_loss=4975289320300.441, outer_radius=250.0381603296059, cable_length=3788098763391.1665)


In [9]:
i1 = 909.32
i2 = 34.64
u1 = 0.4
u2 = 10.5
s1 = 1.73 * u1 * i1
ratio = i1/i2
print(s1)
print(10500/262/1.73)
print(400/10/1.73)

629.24944
23.16551206812867
23.121387283236995


In [10]:
jlv=i1*10/8/342.5
print(jlv)

3.3186861313868614


In [11]:
print(i2*262/13.5/355)


1.8937256129368805
